In [1]:
from datasets import load_dataset
dataset = load_dataset("wikimedia/wikipedia", "20231101.ms")['train'] # you can use other version of this dataset

In [2]:
from text_normalize import normalize_text

normalize_text('hello')

`openai-whisper` is not available, native whisper processor is not available, will use huggingface processor instead.
`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.
/home/husein/.local/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/.local/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/.local/lib/python3.10/site-packages/malaya/normalizer/rules.py:204: FutureWarning: Possible nested set at position 42
  k.lower(): re.compile(_expressions[k]) for k, v in _expressions.items()
/home/husein/.local/lib/python3.10/site-packages/malaya/normalizer/rules.py:204: FutureWarning: Possible nested set at position 3
  k.lower(): re.compile(_expressions[k]) for k, v in _expressions.items()


'hello'

In [3]:
root_directory = "./wiki_clean"

In [45]:
import os
num_shards = 1000

def process_shard(i):
    directory = root_directory + "/shard_" + str(i)
    if os.path.exists(directory):
        print("Shard %d already exists!" % i)
        return
    print('Processing shard %d ...' % i)
    shard = dataset.shard(num_shards=num_shards, index=i)
    processed_dataset = shard.map(lambda t: {'normalize': normalize_text(t['text'])}, remove_columns=['text'])
    if not os.path.exists(directory):
        os.makedirs(directory)
    processed_dataset.save_to_disk(directory)

In [46]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

In [48]:
max_workers = 20

with ProcessPool(max_workers=max_workers) as pool:
    pool.map(process_shard, range(num_shards))

In [52]:
from datasets import load_from_disk, concatenate_datasets

output = [dI for dI in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory,dI))]
datasets = []
for o in output:
    directory = root_directory + "/" + o
    try:
        shard = load_from_disk(directory)
        datasets.append(shard)
    except:
        continue

In [53]:
dataset = concatenate_datasets(datasets)
dataset.save_to_disk('for-tokenizer')

Saving the dataset (0/1 shards):   0%|          | 0/366783 [00:00<?, ? examples/s]

In [6]:
dataset[-1]

{'id': '696002',
 'url': 'https://ms.wikipedia.org/wiki/Kvelia',
 'title': 'Kvelia',
 'normalize': 'Kvelia merupakan sebuah kampung yang terletak di dalam negara Norway . Lihat juga Senarai kampung di Norway Rujukan Senarai kampung di Norway'}

In [7]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers.trainers import WordPieceTrainer
from tokenizers.trainers import WordLevelTrainer
from tqdm import tqdm

tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(vocab_size = 300000, show_progress=True, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [8]:
def batch_iterator():
    batch_size = 1000
    for batch in tqdm(dataset.iter(batch_size=batch_size)):
        yield batch["normalize"]

tokenizer.train_from_iterator(batch_iterator(), length=len(dataset), trainer = trainer)

367it [00:10, 33.52it/s]


In [9]:
tokenizer.save("tokenizer.json")

In [10]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")
tokenizer.bos_token="[CLS]"
tokenizer.eos_token="[SEP]"
tokenizer.sep_token="[SEP]"
tokenizer.cls_token="[CLS]"
tokenizer.unk_token="[UNK]"
tokenizer.pad_token="[PAD]"
tokenizer.mask_token="[MASK]"        
        
tokenizer._bos_token="[CLS]"
tokenizer._eos_token="[SEP]"
tokenizer._sep_token="[SEP]"
tokenizer._cls_token="[CLS]"
tokenizer._unk_token="[UNK]"
tokenizer._pad_token="[PAD]"
tokenizer._mask_token="[MASK]"  

In [11]:
tokenizer.push_to_hub('mesolitica/PL-BERT-MS')

tokenizer.json:   0%|          | 0.00/7.51M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/PL-BERT-MS/commit/20f421ae5644e730387b49d37cab90140988b2dd', commit_message='Upload tokenizer', commit_description='', oid='20f421ae5644e730387b49d37cab90140988b2dd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/PL-BERT-MS', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/PL-BERT-MS'), pr_revision=None, pr_num=None)

In [12]:
dataset[0]['normalize']

'Pelabuhan Kerteh dibina di Kerteh , Terengganu bagi menampung keperluan penghantaran petroleum dan gas asli yang digali di luar pantai Terengganu dan yang ditelah diproses di kawasan daratan Kerteh . Pelabuhan di Malaysia'

In [13]:
len(tokenizer)

300000

In [14]:
tokenizer.tokenize('hello.')

['hello', '.']

In [15]:
tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=300000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)